In [1]:
import tensorflow as tf
import matplotlib 
import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt

%matplotlib inline

/anaconda/envs/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/anaconda/envs/tensorflow/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [12]:
# Training Parameters
learning_rate = 0.01
iterations = 30000
batch_size = 256


# Network Parameters
num_hidden_1 = 240 # 1st layer num features
num_hidden_2 = 200 # 2nd layer num features
num_input = 280 # Flattened first layer

# tf Graph input 
X = tf.placeholder("float", [None, num_input])
labels = tf.placeholder("float", [None,20])
y = labels[:,3:4]

weights = {
    'encoder_h1': tf.Variable(tf.random_normal([num_input, num_hidden_1])),
    'encoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_hidden_2])),
    'decoder_h1': tf.Variable(tf.random_normal([num_hidden_2, num_hidden_1])),
    'decoder_h2': tf.Variable(tf.random_normal([num_hidden_1, num_input])),
}


# Building the encoder
def encoder(x):
    # Encoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.matmul(x, weights['encoder_h1']))
    # Encoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.matmul(layer_1, weights['encoder_h2']))
    return layer_2


# Building the decoder
def decoder(x):
    # Decoder Hidden layer with sigmoid activation #1
    layer_1 = tf.nn.sigmoid(tf.matmul(x, weights['decoder_h1']))
    # Decoder Hidden layer with sigmoid activation #2
    layer_2 = tf.nn.sigmoid(tf.matmul(layer_1, weights['decoder_h2']))
    return layer_2

def get_data(folder_path):
    """ Read the dataset data set and split them into training and test sets """
    train = h5py.File(folder_path+'/train.h5','r')
    test = h5py.File(folder_path+'/test.h5','r')
    return train["data"][()], np.concatenate((train["data"],test["data"])), train["label"][()], np.concatenate((train["label"],test["label"]))



In [13]:
# Construct model
encoder_op = encoder(X)
decoder_op = decoder(encoder_op)


In [16]:
#Load data
train_X, test_X, train_y, test_y = get_data("level_13")

y_pred = decoder_op


In [17]:
# Define loss and optimizer, minimize the squared error

loss = tf.reduce_mean(tf.pow(y - y_pred, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(loss)


In [18]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

In [19]:
with tf.Session() as sess:

    # Run the initializer
    sess.run(init)
    # Training
    for i in range(1, iterations+1):
        # Prepare Data
        batch_x = train_X[i*:,]

        # Run optimization op (backprop) and cost op (to get loss value)
        _, l = sess.run([optimizer, loss], feed_dict={X: batch_x})
        # Display logs per step
        if i % display_step == 0 or i == 1:
            print('Step %i: Minibatch Loss: %f' % (i, l))


ValueError: Cannot feed value of shape (5143, 14, 20) for Tensor 'Placeholder_1:0', which has shape '(?, 280)'